<div align="right" style="text-align: right"><i>Peter Norvig, Feb 2020</i></div>

# CrossProduct Puzzle

The 538 Riddler [poses a type of puzzle](https://fivethirtyeight.com/features/can-you-cross-like-a-boss/) called ***CrossProduct***, which works like this:
>  *Replace each "?" in the table with a single digit so that the product of the digits in each row equals the number to the right of the row, and the product of the digits in each column equals the number above the column.*
>      
> |  6615 | 15552 | &nbsp; 420  |   |
|-------|-------|-------|---|
|    ?  |    ?  |    ?  |**210**|
|    ?  |    ?  |    ?  |**144**|
|    ?  |    ?  |    ?  |**54**|
|    ?  |    ?  |    ?  |**135**|
|    ?  |    ?  |    ?  |**4**|
|    ?  |    ?  |    ?  |**49**|
>
> *This is the solution:*
>
>|6615|15552|&nbsp; 420||
|---|---|---|---|
|7|6|5|**210**|
|9|8|2|**144**|
|3|9|2|**54**|
|5|9|3|**135**|
|1|4|1|**4**|
|7|1|7|**49**|





     
# Data type definitions
     
Here are the data types we will use in trying to solve CrossProduct puzzles: 
- `Digit`: a single digit, from 1 to 9 (but not 0).
- `Row`: a sequence of digits that forms a row in the table, e.g. `(7, 6, 5)`.
- `Table`: a table of digits that fill in for the "?"s; a list of rows, e.g. `[(7, 6, 5), (9, 8, 2), ...]`.
- `Products`: a list of the numbers that corresponding digits must multiply to, e.g. in the puzzle above, `[6615, 15552, 420]` for the column products, and `[210, 144, 54, 135, 4, 49]` for the row products.
- `Puzzle`: a puzzle to be solved, as defined by the row products and column products.

In [1]:
from typing import Tuple, List, Set, Iterable, Optional
from numpy  import divide, prod, transpose
from random import randint
from collections import namedtuple

Digit    = int
Row      = Tuple[Digit, ...] 
Table    = List[Row]       
Products = List[int] 
Puzzle   = namedtuple('Puzzle', 'row_prods, col_prods')

# The puzzles

Here are the puzzles given by 538 Riddler (they promised one a week for four weeks):

In [2]:
puzzles = (Puzzle([135,  45, 64, 280, 70],      [3000,  3969, 640]),
           Puzzle([210, 144, 54, 135,  4, 49],  [6615, 15552, 420]))

# Filling in one row

- A first step in solving the puzzle is filling in a single row of the table.
- `fill_row(product, k)` will return the set of all `k`-digit tuples whose product is `product`.
- In the non-trivial case, pair every first digit, `d`, that divides the product with every way of filling the rest of the row:

In [3]:
def fill_row(product, k=3) -> Set[Row]:
    "All permutations of `k` digits that multiply to `product`."
    return ({()}   if k == 0 and product == 1 else
            set()  if k == 0 and product != 1 else
            {(d, *rest) for d in range(1, 10)
             if (product / d).is_integer()
             for rest in fill_row(product // d, k - 1)})

For example:

In [4]:
fill_row(210)

{(5, 6, 7), (5, 7, 6), (6, 5, 7), (6, 7, 5), (7, 5, 6), (7, 6, 5)}

In [5]:
fill_row(729)

{(9, 9, 9)}

In [6]:
fill_row(729, 4)

{(1, 9, 9, 9),
 (3, 3, 9, 9),
 (3, 9, 3, 9),
 (3, 9, 9, 3),
 (9, 1, 9, 9),
 (9, 3, 3, 9),
 (9, 3, 9, 3),
 (9, 9, 1, 9),
 (9, 9, 3, 3),
 (9, 9, 9, 1)}

In [7]:
fill_row(7**5, 6)

{(1, 7, 7, 7, 7, 7),
 (7, 1, 7, 7, 7, 7),
 (7, 7, 1, 7, 7, 7),
 (7, 7, 7, 1, 7, 7),
 (7, 7, 7, 7, 1, 7),
 (7, 7, 7, 7, 7, 1)}

# Solving a whole puzzle

- We can now solve a whole puzzle with a simple brute-force strategy:
- For every possible way of filling the first row,  try every way of recursively solving the rest of the puzzle. 
- `solve` finds the first solution to a puzzle. (A well-formed puzzle has exactly one solution, but some might have more or less.)
- `solutions` yields all possible solutions to  a puzzle. There are three main cases to consider:
  - A puzzle with no rows has the empty table, `[]`, as a solution, as long as the column products are all 1.
  - A puzzle with rows might have solutions, as long as the column products are all integers. Call `fill_row` to get all possible ways to fill the first row, and for each one recursively call `solutions` to get all the possible ways of filling the rest of the rows (making sure to pass in an altered `col_prods` where each element is divided by the corresponding element in the first row).
  - Otherwise there are no solutions.

In [8]:
def solve(puzzle) -> Optional[Table]: return next(solutions(puzzle), None)

def solutions(puzzle) -> Iterable[Table]:
    """Yield all tables that solve the puzzle.
    The product of the digits in row r must equal row_prods[r], for all r.
    The product of the digits in column c must equal col_prods[c], for all c."""
    row_prods, col_prods = puzzle
    if not row_prods and all(c == 1 for c in col_prods):
        yield []
    elif row_prods and all(c == int(c) for c in col_prods):
        for row1 in fill_row(row_prods[0], len(col_prods)):
            for rows in solutions(Puzzle(row_prods[1:], divide(col_prods, row1))):
                yield [row1, *rows]

# Solutions

Here are  solutions to the puzzles posed by *The Riddler*:

In [9]:
[solve(p) for p in puzzles]

[[(3, 9, 5), (5, 9, 1), (8, 1, 8), (5, 7, 8), (5, 7, 2)],
 [(7, 6, 5), (9, 8, 2), (3, 9, 2), (5, 9, 3), (1, 4, 1), (7, 1, 7)]]

Those are the correct solutions. However, we could make the solutions prettier:

In [10]:
from IPython.display import Markdown

def pretty(puzzle, table=None) -> str:
    """A puzzle and the filled-in table as a str that will be pretty in Markdown."""
    row_prods, col_prods = puzzle
    table = table or solve(puzzle)
    head  = surround(col_prods + [''])
    dash  = surround(['---'] * (1 + len(col_prods)))
    rest  = [surround(row + (f'**{rp}**',))
             for row, rp in zip(table, row_prods)]
    return '\n'.join([head, dash, *rest])

def surround(items, delim='|') -> str: 
    """Like str.join, but delimiter is outside items as well as between."""
    return delim + delim.join(map(str, items)) + delim

In [11]:
Markdown('\n\n'.join(map(pretty, puzzles)))

|3000|3969|640||
|---|---|---|---|
|3|9|5|**135**|
|5|9|1|**45**|
|8|1|8|**64**|
|5|7|8|**280**|
|5|7|2|**70**|

|6615|15552|420||
|---|---|---|---|
|7|6|5|**210**|
|9|8|2|**144**|
|3|9|2|**54**|
|5|9|3|**135**|
|1|4|1|**4**|
|7|1|7|**49**|

# Making new well-formed puzzles

Can we make new well-formed puzzles (those with exactly one solution)? One approach is to:
- Make a table filled with random digits (`random_table`).
- Make a puzzle from the row and column products of the table (`table_puzzle`).
- Check if each puzzle is `well-formed` (that is, has a single solutions).
- Repeat `ntables` times (`random_puzzles`).

In [12]:
def random_table(nrows, ncols) -> Table:
    "Make a table of random digits of the given size."
    return [tuple(randint(1, 9) for c in range(ncols))
            for r in range(nrows)]

def table_puzzle(table) -> Puzzle:
    "Given a table, compute the puzzle it is a solution for."
    return Puzzle([prod(row) for row in table], 
                  [prod(col) for col in transpose(table)])

def well_formed(puzzle) -> bool: 
    "Does the puzzle have exactly one solution?"
    S = solutions(puzzle)
    first, second = next(S, None), next(S, None)
    return first is not None and second is None

def random_puzzles(ntables, nrows=6, ncols=3) -> Iterable[Puzzle]:
    "Generate `ntables` random tables and return the well-formed puzzles from them."
    puzzles = (table_puzzle(random_table(nrows, ncols)) 
               for _ in range(ntables))
    return [puzzle for puzzle in puzzles if well_formed(puzzle)]

In [13]:
random_table(nrows=2, ncols=3)

[(4, 2, 5), (6, 7, 9)]

In [14]:
table_puzzle(_)

Puzzle(row_prods=[40, 378], col_prods=[24, 14, 45])

In [15]:
well_formed(_)

True

In [16]:
list(random_puzzles(200, 5, 3))

[Puzzle(row_prods=[144, 42, 189, 320, 56], col_prods=[16128, 784, 1620]),
 Puzzle(row_prods=[243, 192, 3, 147, 315], col_prods=[567, 10584, 1080]),
 Puzzle(row_prods=[60, 12, 49, 105, 150], col_prods=[945, 2800, 210]),
 Puzzle(row_prods=[240, 162, 50, 98, 120], col_prods=[5376, 1575, 2700]),
 Puzzle(row_prods=[15, 14, 216, 315, 512], col_prods=[1008, 1120, 6480]),
 Puzzle(row_prods=[90, 200, 80, 45, 343], col_prods=[22680, 1120, 875]),
 Puzzle(row_prods=[10, 147, 112, 192, 56], col_prods=[5376, 336, 980]),
 Puzzle(row_prods=[441, 160, 100, 175, 12], col_prods=[3360, 900, 4900]),
 Puzzle(row_prods=[320, 12, 45, 378, 18], col_prods=[150, 15552, 504]),
 Puzzle(row_prods=[343, 120, 90, 35, 84], col_prods=[945, 1344, 8575]),
 Puzzle(row_prods=[150, 36, 15, 98, 75], col_prods=[504, 2625, 450]),
 Puzzle(row_prods=[256, 162, 147, 10, 504], col_prods=[3360, 3888, 2352]),
 Puzzle(row_prods=[24, 45, 60, 252, 315], col_prods=[1512, 19440, 175]),
 Puzzle(row_prods=[160, 135, 147, 50, 90], col_prods

In [17]:
list(random_puzzles(200, 6, 3))

[Puzzle(row_prods=[210, 56, 75, 40, 180, 189], col_prods=[875, 51030, 26880]),
 Puzzle(row_prods=[324, 245, 432, 200, 50, 6], col_prods=[1680, 12600, 97200])]

In [18]:
list(random_puzzles(200, 7, 3))

[]

I've done this several times, and it looks like about 5% of the random 5×3 tables, 2% of the random 6×3 tables, and less than 1% of the random 7×3 tables are well-formed puzzles. 

How fast is it to find a solution? Very fast for small puzzles:

In [19]:
%time [solve(p) for p in puzzles]

CPU times: user 3.45 ms, sys: 802 µs, total: 4.26 ms
Wall time: 3.54 ms


[[(3, 9, 5), (5, 9, 1), (8, 1, 8), (5, 7, 8), (5, 7, 2)],
 [(7, 6, 5), (9, 8, 2), (3, 9, 2), (5, 9, 3), (1, 4, 1), (7, 1, 7)]]

For larger puzzles it is slower. For example, a 10 x 5 puzzle  usually takes between 1/10 second and 10 seconds:

In [20]:
p = table_puzzle(random_table(10, 5))
%time t = solve(p)

CPU times: user 124 ms, sys: 3.91 ms, total: 128 ms
Wall time: 126 ms


In [21]:
Markdown(pretty(p, t))

|1814400|645120|77157360|829440|1905120||
|---|---|---|---|---|---|
|3|8|6|6|7|**6048**|
|3|4|5|3|4|**720**|
|2|9|7|6|9|**6804**|
|6|4|9|4|5|**4320**|
|4|2|7|5|8|**2240**|
|4|7|6|8|3|**4032**|
|5|5|9|2|7|**3150**|
|7|4|6|3|3|**1512**|
|6|1|2|8|1|**96**|
|5|2|9|1|3|**270**|

# Tests

A suite of unit tests:

In [22]:
def test():
    "Test suite for CrossProduct functions."
    assert fill_row(1, 0)  == {()}
    assert fill_row(2, 0)  == set()
    assert fill_row(9, 1)  == {(9,)}
    assert fill_row(10, 1) == set()
    assert fill_row(73, 3) == set()
    
    assert solve(Puzzle([], []))   == []
    assert solve(Puzzle([], [1]))  == []
    assert solve(Puzzle([], [2]))  == None
    assert solve(Puzzle([5], [5])) == [(5,)]
    assert solve(Puzzle([0], [0])) == None # Maybe should allow zero as a digit?
    assert solve(Puzzle([2, 12], [3, 8])) == [(1, 2), (3, 4)]

    assert fill_row(729, 3) == {(9, 9, 9)} # Unique fill
    
    assert fill_row(729, 4) == {
     (1, 9, 9, 9),
     (3, 3, 9, 9),
     (3, 9, 3, 9),
     (3, 9, 9, 3),
     (9, 1, 9, 9),
     (9, 3, 3, 9),
     (9, 3, 9, 3),
     (9, 9, 1, 9),
     (9, 9, 3, 3),
     (9, 9, 9, 1)}
    
    # 72 has the most ways to fill a 3-digit row
    assert max(range(1, 100), key=lambda n: len(fill_row(n, 3))) == 72
    assert fill_row(72, 3)  == { 
     (1, 8, 9),
     (1, 9, 8),
     (2, 4, 9),
     (2, 6, 6),
     (2, 9, 4),
     (3, 3, 8),
     (3, 4, 6),
     (3, 6, 4),
     (3, 8, 3),
     (4, 2, 9),
     (4, 3, 6),
     (4, 6, 3),
     (4, 9, 2),
     (6, 2, 6),
     (6, 3, 4),
     (6, 4, 3),
     (6, 6, 2),
     (8, 1, 9),
     (8, 3, 3),
     (8, 9, 1),
     (9, 1, 8),
     (9, 2, 4),
     (9, 4, 2),
     (9, 8, 1)}
    
    assert solve(Puzzle([210, 144, 54, 135, 4, 49], [6615, 15552, 420])) == [
        (7, 6, 5), 
        (9, 8, 2), 
        (3, 9, 2), 
        (5, 9, 3), 
        (1, 4, 1), 
        (7, 1, 7)]
    
    assert sorted(solutions(Puzzle([8, 8, 1], [8, 8, 1]))) == [ # Multi-solution puzzle
        [(1, 8, 1), 
         (8, 1, 1), 
         (1, 1, 1)],
        [(2, 4, 1), 
         (4, 2, 1), 
         (1, 1, 1)],
        [(4, 2, 1), 
         (2, 4, 1), 
         (1, 1, 1)],
        [(8, 1, 1), 
         (1, 8, 1), 
         (1, 1, 1)]]
    
    assert not list(solutions(Puzzle([8, 8, 1], [8, 8, 2]))) # Unsolvable puzzle
    
    assert solve(Puzzle([1470, 720, 270, 945, 12, 343], 
                        [6615, 15552, 420, 25725])) == [ # 4 column puzzle
        (7, 6, 5, 7),
        (9, 8, 2, 5),
        (3, 9, 2, 5),
        (5, 9, 3, 7),
        (1, 4, 1, 3),
        (7, 1, 7, 7)]
    
    puzz  = Puzzle([6, 120, 504], [28, 80, 162])
    table = [(1, 2, 3), 
             (4, 5, 6), 
             (7, 8, 9)]
    assert solve(puzz) == table
    assert table_puzzle(table) == puzz
    assert well_formed(puzz)
    
    assert not well_formed(Puzzle([7, 7], [7, 7]))
    assert well_formed(Puzzle([64, 224, 189, 270, 405, 144, 105], 
                              [308700, 12960, 1119744]))
    
    assert surround((1, 2, 3)) == '|1|2|3|'
    
    return True
    
test()

True